# Сортировка данных

## Метод pl.DataFrame.sort

Метод `sort()` позволяет отсортировать строки *DataFrame* по значениям в одной или нескольких колонках. Результатом будет новый *DataFrame*, в котором строки упорядочены в соответствии с заданными критериями.

Параметры метода:
- `by` - колонка или выражение, по которому сортируем;
- `more_by` - дополнительные колонки для сортировки (если нужно сортировать по нескольким полям);
- `descending` - если True, то сортировка идёт по убыванию значений (по умолчанию False);
- `nulls_last` - если True, то все значения null будут в конце (по умолчанию False);
- `multithreaded` - если True, то используется многопоточность, т.е. ускоряет сортировку (по умолчанию True);
- `maintain_order` - если True, то при равных значениях сохраняет исходный порядок (по умолчанию False).

Примеры использования:

In [1]:
# Импорт либы
import polars as pl

# Формируем DataFrame
df = pl.DataFrame({
    "a": [1, 2, None],
    "b": [6.0, 5.0, 4.0],
    "c": ["a", "c", "b"],
})

# Сортировка по одной колонке
df.sort("a")

a,b,c
i64,f64,str
null,4.0,"""b"""
1,6.0,"""a"""
2,5.0,"""c"""


По умолчанию: сортировка по возрастанию, null — в начале. 

Можно сортировать не только по колонкам, но и по выражениям:

In [2]:
df.sort(pl.col("a") + pl.col("b") * 2, nulls_last=True) # Здесь сортируем по формуле: a + 2*b; null-значения в конце.

a,b,c
i64,f64,str
2,5.0,"""c"""
1,6.0,"""a"""
null,4.0,"""b"""


Сортировка по нескольким колонкам:

In [3]:
df.sort(["c", "a"], descending=True, nulls_last=True) # Сначала сортируем по c по убыванию, затем по a по убыванию.

a,b,c
i64,f64,str
2,5.0,"""c"""
null,4.0,"""b"""
1,6.0,"""a"""


При помощи позиционных аргументов:

In [4]:
df.sort("c", "a", descending=[False, True], nulls_last = True) # По c — по возрастанию, по a — по убыванию. 

a,b,c
i64,f64,str
1,6.0,"""a"""
null,4.0,"""b"""
2,5.0,"""c"""


Метод `sort()` — гибкий и мощный инструмент для упорядочивания данных. Он поддерживает:
- сортировку по колонкам и выражениям,
- множественные поля с разным направлением,
- контроль положения null-значений,
- высокую производительность за счёт многопоточности.

Сортировка — дорогая операция, особенно на больших данных. Используйте только когда нужно.

## Метод polars.DataFrame.top_k

Часто в анализе данных нужно ответить на вопросы вроде:
- Какие 5 товаров самые продаваемые?
- Кто из студентов получил самые высокие баллы?
- Какие 10 фильмов имеют лучший рейтинг?
  
Метод `top_k()` позволяет быстро получить k строк с наибольшими значениями в указанных колонках — без полной сортировки всего *DataFrame*. 

Параметры метода `top_k()`:
- `k` - количество строк, которые нужно вернуть;
- `by` - Колонка(и), по которым определяется «топ» (можно использовать выражения);
- `reverse` - если True, то возвращает наименьшее значение, иначе по наибольшим (по умолчанию False).

Сформируем *DataFrame* по фильмам:

In [5]:
df_movies = pl.DataFrame({
    "title": [
        "Inception", "Interstellar", "The Dark Knight",
        "Tenet", "Dunkirk", "Memento", "Oppenheimer",
        "The Prestige", "Batman Begins", "Insomnia"
    ],
    "year": [2010, 2014, 2008, 2020, 2017, 2000, 2023, 2006, 2005, 2002],
    "rating": [8.8, 8.6, 9.0, 7.3, 8.6, 8.5, 8.6, 8.5, 8.2, 8.1],
    "box_office_mln": [836, 677, 1005, 365, 527, 40, 974, 109, 373, 113],
    "genre": ["Sci-Fi", "Sci-Fi", "Action", "Action", "War", "Thriller", "Biography", "Drama", "Action", "Thriller"]
})

print(df_movies)

shape: (10, 5)
┌─────────────────┬──────┬────────┬────────────────┬───────────┐
│ title           ┆ year ┆ rating ┆ box_office_mln ┆ genre     │
│ ---             ┆ ---  ┆ ---    ┆ ---            ┆ ---       │
│ str             ┆ i64  ┆ f64    ┆ i64            ┆ str       │
╞═════════════════╪══════╪════════╪════════════════╪═══════════╡
│ Inception       ┆ 2010 ┆ 8.8    ┆ 836            ┆ Sci-Fi    │
│ Interstellar    ┆ 2014 ┆ 8.6    ┆ 677            ┆ Sci-Fi    │
│ The Dark Knight ┆ 2008 ┆ 9.0    ┆ 1005           ┆ Action    │
│ Tenet           ┆ 2020 ┆ 7.3    ┆ 365            ┆ Action    │
│ Dunkirk         ┆ 2017 ┆ 8.6    ┆ 527            ┆ War       │
│ Memento         ┆ 2000 ┆ 8.5    ┆ 40             ┆ Thriller  │
│ Oppenheimer     ┆ 2023 ┆ 8.6    ┆ 974            ┆ Biography │
│ The Prestige    ┆ 2006 ┆ 8.5    ┆ 109            ┆ Drama     │
│ Batman Begins   ┆ 2005 ┆ 8.2    ┆ 373            ┆ Action    │
│ Insomnia        ┆ 2002 ┆ 8.1    ┆ 113            ┆ Thriller  │
└─────────

Пример 1. Выведем Топ-3 самых высокооценённых фильмов.

In [6]:
df_movies.top_k(k=3, by="rating")

title,year,rating,box_office_mln,genre
str,i64,f64,i64,str
"""The Dark Knight""",2008,9.0,1005,"""Action"""
"""Inception""",2010,8.8,836,"""Sci-Fi"""
"""Interstellar""",2014,8.6,677,"""Sci-Fi"""


Пример 2. Выведем Топ-4 по сборам

In [7]:
df_movies.top_k(4, by="box_office_mln")

title,year,rating,box_office_mln,genre
str,i64,f64,i64,str
"""The Dark Knight""",2008,9.0,1005,"""Action"""
"""Oppenheimer""",2023,8.6,974,"""Biography"""
"""Inception""",2010,8.8,836,"""Sci-Fi"""
"""Interstellar""",2014,8.6,677,"""Sci-Fi"""


Пример 3. Найдём топ-5 фильмов по:
1. Рейтингу (по убыванию)
2. Году выпуска (новее — лучше)

In [8]:
df_movies.top_k(5, by=["rating", "year"])

title,year,rating,box_office_mln,genre
str,i64,f64,i64,str
"""The Dark Knight""",2008,9.0,1005,"""Action"""
"""Inception""",2010,8.8,836,"""Sci-Fi"""
"""Oppenheimer""",2023,8.6,974,"""Biography"""
"""Dunkirk""",2017,8.6,527,"""War"""
"""Interstellar""",2014,8.6,677,"""Sci-Fi"""


Сначала сравнивается rating, при равных — year. 

Пример 4. Получим 3 наименее кассовых фильма.

In [9]:
df_movies.top_k(3, by="box_office_mln", reverse=True)

title,year,rating,box_office_mln,genre
str,i64,f64,i64,str
"""Memento""",2000,8.5,40,"""Thriller"""
"""The Prestige""",2006,8.5,109,"""Drama"""
"""Insomnia""",2002,8.1,113,"""Thriller"""


Пример 5. Получим топ-3 фильма с лучшим соотношением рейтинг/сборы (эффективность):

In [10]:
df_movies.top_k(3, by=pl.col("rating") / pl.col("box_office_mln"))

title,year,rating,box_office_mln,genre
str,i64,f64,i64,str
"""Memento""",2000,8.5,40,"""Thriller"""
"""The Prestige""",2006,8.5,109,"""Drama"""
"""Insomnia""",2002,8.1,113,"""Thriller"""


Важные особенности:
1. null значения игнорируются — топовые строки всегда не-null, если есть такие.
2. порядок не гарантируется — используйте .sort() после, если нужен порядок.